In [65]:
from typing import TypedDict, Annotated, Optional
from langgraph.graph import add_messages, StateGraph, END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver
from uuid import uuid4
from elevenlabs.client import ElevenLabs
import json
import os
import google.generativeai as genai
import pygame
import tempfile
import time

# Load environment variables
load_dotenv()

# Get API keys
gemini_key = os.getenv('GEMINI_API_KEY')
elevenlabs_key = os.getenv('ELEVENLABS_API_KEY')

# Initialize pygame mixer for audio playback
pygame.mixer.init()

# Initialize ElevenLabs client
elevenlabs_client = ElevenLabs(api_key=elevenlabs_key)

genai.configure(api_key=gemini_key)


In [66]:
def text_to_speech(text, voice_id="JBFqnCBsd6RMkjVDRZzb"):
    """Convert text to speech using Eleven Labs and pygame"""
    try:
        # Generate audio using the correct API
        audio = elevenlabs_client.text_to_speech.convert(
            text=text,
            voice_id=voice_id,
            model_id="eleven_multilingual_v2",
            output_format="mp3_44100_128",
        )
        
        # Convert the audio to bytes
        audio_bytes = b"".join(audio)
        
        # Create a temporary file with a unique name
        with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_file:
            temp_file.write(audio_bytes)
            temp_filename = temp_file.name
        
        # Stop any currently playing music
        pygame.mixer.music.stop()
        
        # Load and play the new audio file
        pygame.mixer.music.load(temp_filename)
        pygame.mixer.music.play()
        
        # Wait for playback to finish
        while pygame.mixer.music.get_busy():
            pygame.time.wait(100)
        
        # Clean up the temporary file
        try:
            os.unlink(temp_filename)
        except OSError:
            pass  # File might already be deleted
            
        return True
    except Exception as e:
        print(f"Error generating speech: {e}")
        return False

print("✅ Text-to-speech function created!")


✅ Text-to-speech function created!


In [67]:
# Test the updated audio functionality
print("🎵 Testing Updated Audio System...")
test_text = "Hello! This is a test of the updated text-to-speech functionality for our debate agents."
success = text_to_speech(test_text)
if success:
    print("✅ Audio test successful!")
else:
    print("❌ Audio test failed!")


🎵 Testing Updated Audio System...
✅ Audio test successful!


In [68]:
# Clean up any existing temp_audio.mp3 file
try:
    if os.path.exists("temp_audio.mp3"):
        os.unlink("temp_audio.mp3")
        print("Cleaned up existing temp_audio.mp3")
except OSError as e:
    print(f"Could not clean up temp_audio.mp3: {e}")

# Add audio control functions
def stop_audio():
    """Stop any currently playing audio"""
    pygame.mixer.music.stop()

def pause_audio():
    """Pause currently playing audio"""
    pygame.mixer.music.pause()

def resume_audio():
    """Resume paused audio"""
    pygame.mixer.music.unpause()

print("✅ Audio control functions added!")


✅ Audio control functions added!


In [ ]:
# Test the full debate system with audio
print("🚀 Testing Full Debate System with Audio...")
print("=" * 80)

# Test with a shorter debate to verify audio works
test_claim = "Artificial Intelligence will improve education"
print(f"🎯 DEBATE TOPIC: {test_claim}")
print("🎧 Audio will play for each agent's response...")
print("=" * 80)

# Run a short test debate
result = test_debate(test_claim, max_rounds=2)


🚀 Testing Full Debate System with Audio...
🎯 DEBATE TOPIC: Artificial Intelligence will improve education
🎧 Audio will play for each agent's response...
🎯 DEBATE TOPIC: Artificial Intelligence will improve education
📊 Rounds: 2
🎧 Audio will play for each agent's response...

🎤 PROPONENT (Round 1):
--------------------------------------------------
Thank you. I am ready to begin.

The proposition that Artificial Intelligence will improve education is not a speculative fantasy; it is an impending reality. To understand the transformative potential of AI in our classrooms, we can look to the clear and powerful precedent being set in another critical, human-centric field: healthcare.

As the provided evidence consistently highlights, "Artificial intelligence will significantly improve healthcare outcomes by enabling early disease detection, personalized treatment plans, and reducing medical errors through advanced diagnostic tools" (Evidence 1-5). The core principles driving this revolutio

✅ Enhanced voice options added!
Available voices:
  - proponent: JBFqnCBsd6RMkjVDRZzb
  - conponent: EXAVITQu4vr4xnSDxMaL
  - moderator: pNInz6obpgDQGcFmaJgB
  - narrator: AZnzlk1XvdvUeBnXmlld


In [ ]:
# Initialize Gemini model
model = genai.GenerativeModel('gemini-2.5-pro')
print("Gemini model initialized successfully!")


Gemini model initialized successfully!


🎵 Testing Different Voice Options...

🔊 Testing proponent voice (JBFqnCBsd6RMkjVDRZzb):
✅ proponent voice test successful!
------------------------------

🔊 Testing conponent voice (EXAVITQu4vr4xnSDxMaL):
✅ conponent voice test successful!
------------------------------

🔊 Testing moderator voice (pNInz6obpgDQGcFmaJgB):
✅ moderator voice test successful!
------------------------------

🔊 Testing narrator voice (AZnzlk1XvdvUeBnXmlld):
✅ narrator voice test successful!
------------------------------


In [ ]:
from typing import List, Dict


In [ ]:
class State(TypedDict):
    claim: str
    messages: Annotated[list, add_messages]  # LangGraph message system
    round_number: int
    max_rounds: int
    conversation_history: list

In [ ]:
class RAGDebateAgent:

    def __init__(self, name, position, model, db_vector):
        self.name = name
        self.position = position
        self.model = model
        self.db_vector = db_vector

    def retrieve_evidence(self, query, top_k=5):
        """Retrieve evidence from vector database"""
        evidence = []
        
        if self.db_vector:
            try:
                vector_results = self.db_vector.search(query, top_k)
                for docs, score in vector_results:
                    evidence.append({
                        "content": docs.page_content, 
                        "source": docs.metadata.get('source', 'Unknown'),
                        "score": score
                    })
            except Exception as e:
                print(f"Vector DB error: {e}")
                
        return evidence   

    def generate_debate_response(self, state: State): 
        """Generate a debate response using evidence"""
        search_query = f"{state['claim']} {self.position} arguments"
        evidence = self.retrieve_evidence(search_query, top_k=5)

        evidence_context = ""
        for i, ev in enumerate(evidence, 1):
            evidence_context += f"Evidence {i} ({ev['source']}): {ev['content']}...\n"

        history_context = ""
        if state['messages']:
            history_context = "Previous exchanges:\n"
            for msg in state['messages']:
                history_context += f"{msg['role']}: {msg['content']}...\n"

        prompt = f"""
You are {self.name}, a {self.position}ponent in a debate.

DEBATE TOPIC: "{state['claim']}"

ROUND: {state['round_number']}/{state['max_rounds']}

{history_context}

EVIDENCE TO SUPPORT YOUR {self.position.upper()} POSITION:
{evidence_context}

INSTRUCTIONS:
1. Use the evidence above to support your {self.position} argument.
2. Address any points made by the opponent.
3. Be persuasive but respectful.
4. Keep response concise and cite evidence where possible.

Your {self.position} response:
"""

        try:
            response = self.model.generate_content(prompt)
            return response.text
        except Exception as e:
            return f"Error generating response: {e}"    

In [ ]:
graph_builder = StateGraph(State)

In [ ]:
# Import VectorDB class
from vector_db import VectorDB

#initalize vector db
pinecone_db = VectorDB("rag-debate-index")

pro_agent = RAGDebateAgent(
    name="Proponent",
    position="pro",
    model = model,
    db_vector = pinecone_db
)

con_agent = RAGDebateAgent(
    name="Conponent",
    position="con",
    model = model,
    db_vector = pinecone_db
)

print("Multi Agents intialized")

Connected to Pinecone index 'rag-debate-index' with OpenAI embeddings
Multi Agents intialized


In [ ]:




#agent node functions
def pro_agent_node(state):
    response = pro_agent.generate_debate_response(state)
    
    # Print and speak the response
    print(f"\n🎤 PROPONENT (Round {state['round_number']}):")
    print("-" * 50)
    print(response)
    print()
    
    # Convert to speech with enhanced audio system
    print("🔊 Playing Proponent's response...")
    play_agent_audio(response, "proponent")
    
    # Update conversation history
    new_entry = {
        "speaker": "Proponent",
        "response": response,
        "round": state["round_number"]
    }

    updated_history = state["conversation_history"] + [new_entry]

    return {
        "conversation_history": updated_history,
        "round_number": state["round_number"] + 1
    }

def con_agent_node(state):
    response = con_agent.generate_debate_response(state)

    # Print and speak the response
    print(f"\n🎤 CONPONENT (Round {state['round_number']}):")
    print("-" * 50)
    print(response)
    print()
    
    # Convert to speech with enhanced audio system
    print("🔊 Playing Conponent's response...")
    play_agent_audio(response, "conponent")
    
    # Update conversation history
    new_entry = {
        "speaker": "Conponent",
        "response": response,
        "round": state["round_number"]
    }

    updated_history = state["conversation_history"] + [new_entry]

    return {
        "conversation_history": updated_history,
        "round_number": state["round_number"] + 1
    }

def should_continue(state: State):
    """Determine if debate should continue"""
    if state["round_number"] >= state["max_rounds"]:
        return "end"
    elif state["round_number"] % 2 == 1:  # Odd rounds go to pro_agent
        return "pro_agent"
    else:  # Even rounds go to con_agent
        return "con_agent"

print("✅ Agent node functions defined!")

✅ Agent node functions defined!


In [ ]:
graph_builder = StateGraph(State)

In [ ]:
graph_builder = StateGraph(State)

#add nodes
graph_builder.add_node("pro_agent", pro_agent_node)
graph_builder.add_node("con_agent", con_agent_node)

# Add conditional routing from pro_agent
graph_builder.add_conditional_edges(
    "pro_agent",
    should_continue,
    {
        "con_agent": "con_agent",
        "end": END
    }
)

graph_builder.add_conditional_edges(
    "con_agent", 
    should_continue,
    {
        "pro_agent": "pro_agent",
        "end": END
    }
)

graph_builder.set_entry_point("pro_agent")


compiled_graph = graph_builder.compile()

print("✅ Multi-agent debate graph compiled!")

✅ Multi-agent debate graph compiled!


In [ ]:
# Add some test data to the vector database
test_documents = [
    "Artificial intelligence will significantly improve healthcare outcomes by enabling early disease detection, personalized treatment plans, and reducing medical errors through advanced diagnostic tools.",
    "AI in healthcare raises serious privacy concerns as it requires access to sensitive patient data, potentially leading to data breaches and unauthorized access to personal medical information.",
    "Machine learning algorithms can help doctors make more accurate diagnoses by analyzing vast amounts of medical data and identifying patterns that humans might miss.",
    "The implementation of AI in healthcare is extremely expensive, requiring significant infrastructure investments that many hospitals and clinics cannot afford, potentially widening healthcare disparities.",
    "AI-powered robotic surgery can perform complex procedures with greater precision than human surgeons, reducing recovery times and improving patient outcomes.",
    "AI systems in healthcare lack transparency and explainability, making it difficult for doctors and patients to understand how decisions are made, which could lead to medical malpractice issues.",
    "Telemedicine powered by AI can provide healthcare access to remote and underserved populations, democratizing medical care and reducing healthcare costs.",
    "AI algorithms can perpetuate existing biases in healthcare data, leading to discriminatory treatment recommendations for certain demographic groups.",
    "Predictive analytics in AI can help prevent hospital readmissions by identifying high-risk patients and providing proactive care interventions.",
    "The over-reliance on AI in healthcare could lead to deskilling of medical professionals and reduce the human element that is crucial for patient care and empathy."
]

test_metadata = [
    {"source": "healthcare_ai_pros", "topic": "AI Healthcare Benefits"},
    {"source": "healthcare_ai_cons", "topic": "AI Healthcare Privacy"},
    {"source": "healthcare_ai_pros", "topic": "AI Diagnostic Accuracy"},
    {"source": "healthcare_ai_cons", "topic": "AI Healthcare Costs"},
    {"source": "healthcare_ai_pros", "topic": "AI Surgical Precision"},
    {"source": "healthcare_ai_cons", "topic": "AI Transparency Issues"},
    {"source": "healthcare_ai_pros", "topic": "AI Telemedicine"},
    {"source": "healthcare_ai_cons", "topic": "AI Bias Concerns"},
    {"source": "healthcare_ai_pros", "topic": "AI Predictive Analytics"},
    {"source": "healthcare_ai_cons", "topic": "AI Human Element"}
]

# Add documents to vector database
pinecone_db.add_documents(test_documents, test_metadata)
print("✅ Test data added to vector database!")


Added 10 documents to vector store
✅ Test data added to vector database!


In [ ]:
# Test function to run the debate
def test_debate(claim, max_rounds=4):
    """Test the RAG debate system with a given claim"""
    
    # Initialize the state
    initial_state = {
        "claim": claim,
        "messages": [],
        "round_number": 1,
        "max_rounds": max_rounds,
        "conversation_history": []
    }
    
    print(f"🎯 DEBATE TOPIC: {claim}")
    print(f"📊 Rounds: {max_rounds}")
    print("=" * 80)
    print("🎧 Audio will play for each agent's response...")
    print("=" * 80)
    
    try:
        # Run the debate
        result = compiled_graph.invoke(initial_state)
        
        # Display final summary
        print("\n🏁 DEBATE COMPLETED!")
        print("=" * 80)
        print(f"Total exchanges: {len(result['conversation_history'])}")
        print("=" * 80)
            
        return result
        
    except Exception as e:
        print(f"❌ Error running debate: {e}")
        return None

print("✅ Test function created!")


✅ Test function created!


In [ ]:
# Test the RAG debate system
test_claim = "Artificial Intelligence will revolutionize healthcare for the better"

print("🚀 Starting RAG Debate Test...")
print("=" * 80)

# Run the test
result = test_debate(test_claim, max_rounds=4)


🚀 Starting RAG Debate Test...
🎯 DEBATE TOPIC: Artificial Intelligence will revolutionize healthcare for the better
📊 Rounds: 4
🎧 Audio will play for each agent's response...

🎤 PROPONENT (Round 1):
--------------------------------------------------
Good morning. I am here today to affirm the undeniable truth that artificial intelligence will revolutionize healthcare for the better. This is not a distant-future prediction, but a present-day reality. AI is poised to become the most significant advancement in medicine since the discovery of antibiotics, creating a future where healthcare is more precise, proactive, and personalized for every single patient.

My argument is built on the clear and compelling benefits outlined in our evidence.

First, AI grants us the unprecedented ability for **early disease detection**. As our supporting evidence states, AI can analyze complex medical data—from radiological scans to genetic markers—with a speed and accuracy that surpasses human capability 

KeyboardInterrupt: 

In [ ]:
# Test the audio functionality
print("🎵 Testing Eleven Labs audio...")
test_text = "Hello! This is a test of the text-to-speech functionality for our debate agents."
text_to_speech(test_text)
print("✅ Audio test completed!")


🎵 Testing Eleven Labs audio...
Error generating speech: name 'generate' is not defined
✅ Audio test completed!
